In [1]:
import numpy as np
from thefittest.optimizers import GeneticProgramming
from thefittest.tools import donothing
from thefittest.base import FunctionalNode
from thefittest.base import TerminalNode
from thefittest.base import EphemeralNode
from thefittest.base import UniversalSet
from thefittest.tools.operators import Mul
from thefittest.tools.operators import Add
from thefittest.tools.operators import Div
from thefittest.tools.operators import Neg
from thefittest.tools.metrics import coefficient_determination


def generator1():
    return np.round(np.random.uniform(0, 10), 4)


def generator2():
    return np.random.randint(0, 10)


def problem(x):
    return 3*x[:, 0]**2 + 2*x[:,0] + 5


function = problem
left_border = -4.5
right_border = 4.5
sample_size = 300
n_dimension = 1

number_of_iterations = 100
population_size = 500

X = np.array([np.linspace(left_border, right_border, sample_size)
              for _ in range(n_dimension)]).T
y = function(X)


functional_set = [FunctionalNode(Add()),
                  FunctionalNode(Mul()),
                  FunctionalNode(Neg()),
                  FunctionalNode(Div())]


terminal_set = [TerminalNode(X[:, i], f'x{i}') for i in range(n_dimension)]
terminal_set.extend([EphemeralNode(generator1), EphemeralNode(generator2)])
uniset = UniversalSet(functional_set, terminal_set)


def fitness_function(trees):
    fitness = []
    for tree in trees:
        y_pred = tree()*np.ones(len(y))
        fitness.append(coefficient_determination(y, y_pred))
    return np.array(fitness)


model = GeneticProgramming(fitness_function=fitness_function,
                           genotype_to_phenotype=donothing,
                           uniset=uniset,
                           pop_size=population_size,
                           iters=number_of_iterations,
                           show_progress_each=10,
                           minimization=False,
                           keep_history=False)

model.fit()

fittest = model.get_fittest()
genotype, phenotype, fitness = fittest.get()
print('The fittest individ:', phenotype)
print('with fitness', fitness)


0 iteration with fitness = 0.7960953886400608
10 iteration with fitness = 0.9295430818954024
20 iteration with fitness = 0.9295430818954024
30 iteration with fitness = 1.0
40 iteration with fitness = 1.0
50 iteration with fitness = 1.0
60 iteration with fitness = 1.0
70 iteration with fitness = 1.0
80 iteration with fitness = 1.0
90 iteration with fitness = 1.0
The fittest individ: (7 + (((x0 * ((x0 + x0) + x0)) + x0) + (-2 + x0)))
with fitness 1.0
